In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
import torch.nn.functional as F
from torchvision import transforms
import requests

from res_facenet.models import model_920
model920 = model_920()

haar = cv2.CascadeClassifier('./data/haarcascade_frontalface_default.xml')

%matplotlib inline

In [2]:
def face_detect(image):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = haar.detectMultiScale(gray, 2,2)
    
    for x,y,w,h in faces:
        cv2.rectangle(image, (x,y), (x+w,y+h),(0, 255,255),10)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(image)
    
    return pil_image

In [3]:
def verify_face_with_new_anchor(pil_anchor_image, pil_verification_image):
    #prepare preprocess pipeline
    preprocess_pipelines = [transforms.Resize(224), 
                           transforms.CenterCrop(224), 
                           transforms.ToTensor(), 
                           transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                std=[0.229, 0.224, 0.225])]
    transform = transforms.Compose(preprocess_pipelines)
    topil = transforms.ToPILImage()
    totensor = transforms.Compose(preprocess_pipelines[:-1])

    # read the image and transform it into tensor then normalize it with our transform function pipeline
    anchor_image = transform(pil_anchor_image).unsqueeze(0)
    verification_image = transform(pil_verification_image).unsqueeze(0)

    # do forward pass
    anchor_image_embed, verification_image_embed = model920(anchor_image), model920(verification_image)

    # compute the distance using euclidean distance of image embeddings
    euclidean_distance = F.pairwise_distance(anchor_image_embed, verification_image_embed)

    # we use 1.5 threshold to decide whether images are genuine or impostor

    threshold = 1.5

    genuine = euclidean_distance <= threshold

    return genuine.item()

In [4]:
def verify_face(pil_verification_image):
    #prepare preprocess pipeline
    preprocess_pipelines = [transforms.Resize(224), 
                           transforms.CenterCrop(224), 
                           transforms.ToTensor(), 
                           transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                std=[0.229, 0.224, 0.225])]
    transform = transforms.Compose(preprocess_pipelines)
    topil = transforms.ToPILImage()
    totensor = transforms.Compose(preprocess_pipelines[:-1])

    # read the image and transform it into tensor then normalize it with our transform function pipeline
    verification_image = transform(pil_verification_image).unsqueeze(0)

    # do forward pass
    verification_image_embed = model920(verification_image)

    # compute the distance using euclidean distance of image embeddings
    euclidean_distance = F.pairwise_distance(anchor_image_embed, verification_image_embed)

    # we use 1.5 threshold to decide whether images are genuine or impostor

    threshold = 1.5

    genuine = euclidean_distance <= threshold

    return genuine.item()

In [5]:
def create_anchor_image(pil_image):
    anchor_image = transform(pil_image).unsqueeze(0)
    anchor_image_embed = model920(anchor_image)
    return anchor_image_embed

In [6]:
cap = cv2.VideoCapture(0)
anchor_image = Image.open('./data/me.jpg')

frame_limit = 1
frames = 0
while frames < frame_limit:
    ret,frame = cap.read()
    if ret == False:
        print(f"Printed {frames} frames")
        break
        
    frame = face_detect(frame)

    url = 'http://127.0.0.1:5000/verify-with-anchor'
    #print(np.array(frame))
    #print(np.array(anchor_image))
    myobj = {'verificationImage': np.array(frame), 'anchorImage': np.array(anchor_image)}
    print(myobj)

    print(requests.post(url, data = myobj))
    #cv2.imshow('object_detect',np.asarray(frame))
    if cv2.waitKey(40) == 27: # Every 40 milliseconds check if escape key is pressed
        break
    frames += 1
        
cv2.destroyAllWindows()
cap.release()

{'verificationImage': array([[[ 82,  73,  54],
        [ 82,  73,  54],
        [ 77,  70,  51],
        ...,
        [175, 186, 192],
        [180, 191, 193],
        [189, 200, 202]],

       [[ 82,  73,  54],
        [ 81,  71,  53],
        [ 76,  68,  50],
        ...,
        [182, 193, 199],
        [189, 200, 202],
        [190, 201, 203]],

       [[ 79,  72,  53],
        [ 78,  71,  52],
        [ 76,  68,  50],
        ...,
        [187, 196, 199],
        [194, 201, 199],
        [201, 208, 206]],

       ...,

       [[ 47,  37,  24],
        [ 47,  37,  24],
        [ 42,  34,  20],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[ 47,  39,  25],
        [ 46,  38,  24],
        [ 42,  34,  20],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[ 46,  38,  24],
        [ 44,  37,  22],
        [ 41,  33,  19],
        ...,
        [255, 255, 255],
        [255, 255, 255],
  

ChunkedEncodingError: ("Connection broken: ConnectionResetError(54, 'Connection reset by peer')", ConnectionResetError(54, 'Connection reset by peer'))

In [12]:
#create_anchor_image(transform(pil_anchor_image).unsqueeze(0))
cap = cv2.VideoCapture(0)
anchor_image = Image.open('./data/me.jpg')

frame_limit = 10
frames = 0
while frames < frame_limit:
    ret,frame = cap.read()
    if ret == False:
        print(f"Printed {frames} frames")
        break
        
    frame = face_detect(frame)
    print(verify_face_with_new_anchor(anchor_image, frame))
    #cv2.imshow('object_detect',np.asarray(frame))
    if cv2.waitKey(40) == 27: # Every 40 milliseconds check if escape key is pressed
        break
    frames += 1
        
cv2.destroyAllWindows()
cap.release()

False
True
True
True
True
True
False
False
False
False
